---

### **Text Classification by Fine-tuning Language Model**

---

### 1. **Data Loading**
   - Load the dataset (CSV format in this case).
   - Perform exploratory data analysis (EDA) to understand class distributions and data structure.
   - Split the dataset into training and validation sets.

```python
# Install simpletransformers package
!pip install simpletransformers

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)
data = pd.read_csv('text_classification_data.csv')

# Exploratory Data Analysis (EDA)
print(data.info())  # Overview of data structure
print(data['label'].value_counts())  # Class distribution

# Split dataset into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Preparing the data in the correct format for SimpleTransformers
train_df = pd.DataFrame({
    'text': train_data['text'],
    'labels': train_data['label']
})

val_df = pd.DataFrame({
    'text': val_data['text'],
    'labels': val_data['label']
})
```

---

### 2. **Text Processing**
   - Here we clean the text by removing special characters, converting to lowercase, removing numbers, and stripping any extra whitespace.

```python
import re

# Define a function to clean text data
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = text.strip()
    
    return text

# Apply the cleaning function to the dataset
train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

print(train_df.head())
```

---

### 3. **Text Embedding using BERT and RoBERTa**
   - Use BERT and RoBERTa models for embedding the cleaned text. These models automatically tokenize and embed the text.

```python
from simpletransformers.classification import ClassificationModel

# Create a BERT model for text classification
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

# Create a RoBERTa model for text classification
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU
```

---

### 4. **Model Training with BERT and RoBERTa**

#### **Basic Model Training**

#### **Train the BERT Model**
```python
# Train BERT model
bert_model.train_model(train_df)
```

#### **Train the RoBERTa Model**
```python
# Train RoBERTa model
roberta_model.train_model(train_df)
```

#### **Model Training with Hyperparameters**
   - Train the models with a set of hyperparameters such as learning rate, batch size, epochs, etc.

```python
from simpletransformers.classification import ClassificationArgs

# Set up model arguments with custom hyperparameters
model_args = ClassificationArgs(
    num_train_epochs=3,       # Start with 3 epochs
    train_batch_size=8,       # Use a batch size of 8
    eval_batch_size=8,        # Same for evaluation
    learning_rate=3e-5,       # Learning rate
    max_seq_length=128,       # Max sequence length
    weight_decay=0.01,        # Weight decay
    warmup_steps=0,           # Optional: adjust based on total steps
    logging_steps=50,         # Log training progress every 50 steps
    save_steps=200,           # Save the model every 200 steps
)

# Train the BERT model with custom hyperparameters
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, args=model_args, use_cuda=False)
bert_model.train_model(train_df)

# Train the RoBERTa model with custom hyperparameters
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=model_args, use_cuda=False)
roberta_model.train_model(train_df)
```

---

### 5. **Evaluation on Validation Set**
   - Evaluate the performance of both BERT and RoBERTa models on the validation set using accuracy, precision, recall, and F1-score.

#### **Evaluate BERT Model**
```python
# Evaluate BERT on validation data
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("BERT Evaluation Results:")
print(result_bert)
```

#### **Evaluate RoBERTa Model**
```python
# Evaluate RoBERTa on validation data
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

print("RoBERTa Evaluation Results:")
print(result_roberta)
```

---

### 6. **Saving the Best Model**
   - Save the best-performing model for later use.

#### **Saving the BERT Model**
```python
bert_model.save_model('bert_best_model')
```

#### **Saving the RoBERTa Model**
```python
roberta_model.save_model('roberta_best_model')
```

---

### 7. **Prediction on Real-World Input**
   - Test the saved model on real-world input data. Preprocess the input text, use the model to predict the class, and output the results.

#### **Prediction Using BERT Model**
```python
# Load the saved BERT model
bert_model = ClassificationModel('bert', 'bert_best_model', use_cuda=False)

# Real-world input text
real_world_text = ["This is a great product!", "I didn't like the service."]

# Predict the class
predictions_bert, _ = bert_model.predict(real_world_text)

print(f"BERT Predictions: {predictions_bert}")
```

#### **Prediction Using RoBERTa Model**
```python
# Load the saved RoBERTa model
roberta_model = ClassificationModel('roberta', 'roberta_best_model', use_cuda=False)

# Real-world input text
real_world_text = ["This is a great product!", "I didn't like the service."]

# Predict the class
predictions_roberta, _ = roberta_model.predict(real_world_text)

print(f"RoBERTa Predictions: {predictions_roberta}")
```

---

In [ ]:
# Install simpletransformers package
!pip install simpletransformers -q  # Added -q for quieter output

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Upload file
from google.colab import files
uploaded = files.upload()  # Upload 'Fashion_Trend_Cloth_Dataset_Unique.csv'

# Load the dataset
data = pd.read_csv('Fashion_Trend_Cloth_Dataset_Unique.csv')

try:
    # Rename columns to match the expected format
    data = data.rename(columns={
        'Input (Clothing Description)': 'text',
        'Output (Category)': 'labels'
    })

    # Exploratory Data Analysis (EDA)
    print("Data Info:")
    print(data.info())  # Overview of data structure

    print("\nLabel Distribution:")
    print(data['labels'].value_counts())  # Distribution of all labels

    # Create a label map from all unique labels in the dataset
    label_map = {label: idx for idx, label in enumerate(data['labels'].unique())}
    print("\nLabel Map:", label_map)  # Show the mapping for verification

    # Split dataset into train and validation sets
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    # Preparing the data in the correct format for SimpleTransformers
    train_df = pd.DataFrame({
        'text': train_data['text'],
        'labels': train_data['labels'].map(label_map)  # Map labels to integers
    })

    val_df = pd.DataFrame({
        'text': val_data['text'],
        'labels': val_data['labels'].map(label_map)  # Map labels to integers
    })

    # Check for unmapped labels (should be 0)
    print("\nMissing labels in train_df:", train_df['labels'].isna().sum())
    print("Missing labels in val_df:", val_df['labels'].isna().sum())

    # Display the first few rows
    print("\nTraining Data:")
    print(train_df.head())

    print("\nValidation Data:")
    print(val_df.head())

except KeyError as e:
    print(f"Error: Column not found - {e}. Please check your CSV column names.")
except Exception as e:
    print(f"An error occurred: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 656.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
   

Saving Fashion_Trend_Cloth_Dataset_Unique.csv to Fashion_Trend_Cloth_Dataset_Unique.csv
Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   labels  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None

Label Distribution:
labels
Vintage              266
Seasonal (Summer)    264
Seasonal (Winter)    248
Trending             222
Name: count, dtype: int64

Label Map: {'Seasonal (Summer)': 0, 'Vintage': 1, 'Seasonal (Winter)': 2, 'Trending': 3}

Missing labels in train_df: 0
Missing labels in val_df: 0

Training Data:
                                                  text  labels
29   A slim red wool sweater, designed for both sty...       2
535  A cropped brown chiffon hoodie, designed for b...       2
695  A flared orange leather t-shirt, stitched with...       2
557  A loose yellow satin blazer, c

In [ ]:
# Part 3: Text Embedding and Classification with BERT and RoBERTa
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import logging
import torch
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download NLTK resources (run once)
nltk.download('stopwords')
nltk.download('wordnet')

# Enhanced text cleaning
stop_words = set(stopwords.words('english')) - {'for', 'with'}  # Keep fashion-relevant stopwords
lemmatizer = WordNetLemmatizer()

def clean_text_efficient(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'(\d+)([-/x])\s*(\d+)', r'\1 \3', text)  # Split sizes (e.g., "12-14" → "12 14")
    text = re.sub(r'[^\w\s\'-]', '', text)  # Keep apostrophes/hyphens
    words = text.split()
    cleaned_words = []
    for i, word in enumerate(words):
        if word.isdigit() and i > 0 and words[i-1] in ['size', 'length']:
            cleaned_words.append(word)  # Keep size/length numbers
        elif not word.isdigit():
            word = lemmatizer.lemmatize(word)  # Lemmatize non-numeric words
            if word not in stop_words:
                cleaned_words.append(word)  # Remove stopwords except 'for', 'with'
    return ' '.join(cleaned_words).strip()

# Load and split data (80-20)
data = pd.read_csv('Fashion_Trend_Cloth_Dataset_Unique.csv')
data = data.rename(columns={'Input (Clothing Description)': 'text', 'Output (Category)': 'labels'})
label_map = {label: idx for idx, label in enumerate(data['labels'].unique())}
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
train_df = pd.DataFrame({'text': train_data['text'], 'labels': train_data['labels'].map(label_map)})
val_df = pd.DataFrame({'text': val_data['text'], 'labels': val_data['labels'].map(label_map)})

# Apply text cleaning
train_df['text'] = train_df['text'].apply(clean_text_efficient)
val_df['text'] = val_df['text'].apply(clean_text_efficient)

# Logging setup
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

print("GPU Available:", torch.cuda.is_available())
print("Running on CPU only")

# Model arguments
model_args = ClassificationArgs(
    num_train_epochs=5,              # More epochs for better learning
    learning_rate=3e-5,              # Slightly higher than 2e-5, balanced for small data
    overwrite_output_dir=True,
    evaluate_during_training=True,
    evaluate_during_training_steps=50,  # Check progress every 50 steps
    train_batch_size=8,              # Smaller batch for CPU and better gradients
    eval_batch_size=8,
    max_seq_length=128,              # Matches your text length stats
    sliding_window=False,            # Avoid truncation issues
    output_dir='outputs_dummy',
    save_steps=0,                    # No intermediate saves
    save_model_every_epoch=False,
    save_eval_checkpoints=False,
    save_best_model=False,
    manual_seed=42,
    logging_steps=10                 # Log training loss to monitor progress
)

num_labels = len(label_map)
print(f"Number of unique labels: {num_labels}")

# BERT Model
bert_model = ClassificationModel(
    'bert', 'bert-base-uncased',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False
)

print("\nTraining BERT Model...")
try:
    bert_result = bert_model.train_model(
        train_df,
        eval_df=val_df,
        acc=accuracy_score
    )
    print("BERT Training Result:", bert_result)
except Exception as e:
    print(f"Error training BERT model: {e}")

print("\nEvaluating BERT Model...")
bert_eval_result, _, _ = bert_model.eval_model(val_df, acc=accuracy_score)
print("BERT Evaluation:", bert_eval_result)

# RoBERTa Model
roberta_model = ClassificationModel(
    'roberta', 'roberta-base',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False
)

print("\nTraining RoBERTa Model...")
try:
    roberta_result = roberta_model.train_model(
        train_df,
        eval_df=val_df,
        acc=accuracy_score
    )
    print("RoBERTa Training Result:", roberta_result)
except Exception as e:
    print(f"Error training RoBERTa model: {e}")

print("\nEvaluating RoBERTa Model...")
roberta_eval_result, _, _ = roberta_model.eval_model(val_df, acc=accuracy_score)
print("RoBERTa Evaluation:", roberta_eval_result)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


GPU Available: True
Running on CPU only
Number of unique labels: 4


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Running Epoch 2 of 5:   0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Part 3: Text Embedding and Classification with BERT and RoBERTa
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score
import logging
import torch

# Set up logging to monitor progress
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Confirm CPU-only execution
print("GPU Available:", torch.cuda.is_available())
print("Running on CPU only (use_cuda=False)")

# Define model arguments for better training on CPU
model_args = ClassificationArgs(
    num_train_epochs=5,              # Increase to 5 for more learning
    learning_rate=2e-5,              # Middle ground from your tries
    overwrite_output_dir=True,
    evaluate_during_training=True,
    evaluate_during_training_steps=50,
    train_batch_size=8,              # Smaller batch for CPU and better gradient updates
    eval_batch_size=8,
    max_seq_length=128,              # Matches your text length stats (max 116)
    output_dir='outputs_dummy',
    save_steps=0,
    save_model_every_epoch=False,
    save_eval_checkpoints=False,
    save_best_model=False,
    manual_seed=42
)

# Number of unique labels from the dataset (from Part 1)
num_labels = len(data['labels'].unique())  # Should be 4 based on your earlier output
print(f"Number of unique labels: {num_labels}")

# Create BERT model for text classification
bert_model = ClassificationModel(
    'bert', 'bert-base-uncased',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False  # Force CPU usage
)

# Create RoBERTa model for text classification
roberta_model = ClassificationModel(
    'roberta', 'roberta-base',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False  # Force CPU usage
)

# Train BERT model
print("\nTraining BERT Model...")
try:
    bert_result = bert_model.train_model(
        train_df,
        eval_df=val_df,
        acc=accuracy_score  # Use accuracy as the evaluation metric
    )
    print("BERT Training Result:", bert_result)
except Exception as e:
    print(f"Error training BERT model: {e}")

# Train RoBERTa model
print("\nTraining RoBERTa Model...")
try:
    roberta_result = roberta_model.train_model(
        train_df,
        eval_df=val_df,
        acc=accuracy_score
    )
    print("RoBERTa Training Result:", roberta_result)
except Exception as e:
    print(f"Error training RoBERTa model: {e}")

# Evaluate models
print("\nEvaluating BERT Model...")
try:
    bert_eval_result, _, _ = bert_model.eval_model(val_df, acc=accuracy_score)
    print("BERT Evaluation:", bert_eval_result)
except Exception as e:
    print(f"Error evaluating BERT model: {e}")

print("\nEvaluating RoBERTa Model...")
try:
    roberta_eval_result, _, _ = roberta_model.eval_model(val_df, acc=accuracy_score)
    print("RoBERTa Evaluation:", roberta_eval_result)
except Exception as e:
    print(f"Error evaluating RoBERTa model: {e}")

GPU Available: False
Running on CPU only (use_cuda=False)
Number of unique labels: 4


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Running Epoch 1 of 5:   0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

def print_detailed_metrics(y_true, y_pred, model_name):
    print(f"\n{model_name} Detailed Classification Report:")
    print(classification_report(y_true, y_pred))

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    metrics_dict = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"\n{model_name} Key Metrics:")
    for metric, value in metrics_dict.items():
        print(f"{metric}: {value:.4f}")

    return metrics_dict

# Evaluate BERT Model
print("\nEvaluating BERT Model with Detailed Metrics...")
bert_result, bert_model_outputs, bert_wrong_predictions = bert_model.eval_model(val_df)
bert_preds = bert_model_outputs.argmax(axis=1)  # Get predicted class indices
bert_metrics = print_detailed_metrics(val_df['labels'], bert_preds, "BERT")

# Evaluate RoBERTa Model
print("\nEvaluating RoBERTa Model with Detailed Metrics...")
roberta_result, roberta_model_outputs, roberta_wrong_predictions = roberta_model.eval_model(val_df)
roberta_preds = roberta_model_outputs.argmax(axis=1)  # Get predicted class indices
roberta_metrics = print_detailed_metrics(val_df['labels'], roberta_preds, "RoBERTa")

# Compare both models
print("\nModel Comparison:")
print(f"BERT Accuracy: {bert_metrics['accuracy']:.4f} vs RoBERTa Accuracy: {roberta_metrics['accuracy']:.4f}")
print(f"BERT F1-score: {bert_metrics['f1_score']:.4f} vs RoBERTa F1-score: {roberta_metrics['f1_score']:.4f}")


Evaluating BERT Model with Detailed Metrics...


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]


BERT Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.23      1.00      0.37        69
           2       0.00      0.00      0.00        89
           3       0.00      0.00      0.00        62

    accuracy                           0.23       300
   macro avg       0.06      0.25      0.09       300
weighted avg       0.05      0.23      0.09       300


BERT Key Metrics:
accuracy: 0.2300
precision: 0.0529
recall: 0.2300
f1_score: 0.0860

Evaluating RoBERTa Model with Detailed Metrics...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]


RoBERTa Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.23      1.00      0.37        69
           2       0.00      0.00      0.00        89
           3       0.00      0.00      0.00        62

    accuracy                           0.23       300
   macro avg       0.06      0.25      0.09       300
weighted avg       0.05      0.23      0.09       300


RoBERTa Key Metrics:
accuracy: 0.2300
precision: 0.0529
recall: 0.2300
f1_score: 0.0860

Model Comparison:
BERT Accuracy: 0.2300 vs RoBERTa Accuracy: 0.2300
BERT F1-score: 0.0860 vs RoBERTa F1-score: 0.0860


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# After evaluation code from previous step...

# Compare models using multiple metrics (prioritizing F1-score, then accuracy)
if bert_metrics['f1_score'] > roberta_metrics['f1_score']:
    best_model = bert_model
    best_model_name = "BERT"
    print("\nBERT model performs better based on F1-score. Saving BERT as best model...")
    best_model.save_model(output_dir='best_model')
elif roberta_metrics['f1_score'] > bert_metrics['f1_score']:
    best_model = roberta_model
    best_model_name = "RoBERTa"
    print("\nRoBERTa model performs better based on F1-score. Saving RoBERTa as best model...")
    best_model.save_model(output_dir='best_model')
else:
    # If F1-scores are equal, check accuracy
    if bert_metrics['accuracy'] > roberta_metrics['accuracy']:
        best_model = bert_model
        best_model_name = "BERT"
        print("\nModels have same F1-score but BERT has slightly better accuracy. Saving BERT as best model...")
        best_model.save_model(output_dir='best_model')
    elif roberta_metrics['accuracy'] > bert_metrics['accuracy']:
        best_model = roberta_model
        best_model_name = "RoBERTa"
        print("\nModels have same F1-score but RoBERTa has slightly better accuracy. Saving RoBERTa as best model...")
        best_model.save_model(output_dir='best_model')
    else:
        # If all metrics are equal, default to BERT
        best_model = bert_model
        best_model_name = "BERT"
        print("\nModels perform identically on all metrics. Defaulting to saving BERT as best model...")
        best_model.save_model(output_dir='best_model')

print(f"\nSaved {best_model_name} as the best performing model in 'best_model' directory")


Models perform identically on all metrics. Defaulting to saving BERT as best model...

Saved BERT as the best performing model in 'best_model' directory


In [ ]:
from simpletransformers.classification import ClassificationModel
import os

# Create directory if it doesn't exist
if not os.path.exists('best_model'):
    os.makedirs('best_model')

# Save the model with all necessary files
bert_model.model.save_pretrained('best_model')
bert_model.tokenizer.save_pretrained('best_model')
print("Model saved successfully with all required files!")

Model saved successfully with all required files!


In [ ]:
from simpletransformers.classification import ClassificationModel

# Define your clothing categories (update these based on your actual labels)
CLOTHING_CATEGORIES = {
    0: "Summer Wear",
    1: "Winter Wear",
    2: "Vintage/Classic",
    3: "Sports/Active",
    4: "Formal Wear"
}

try:
    # 1. Load the trained model
    model = ClassificationModel(
        'bert',
        'best_model',
        use_cuda=False,
        args={'reprocess_input_data': True}
    )
    print("Model loaded successfully!")

    # 2. Example clothing descriptions
    clothing_descriptions = [
        "Lightweight cotton t-shirt with short sleeves perfect for hot weather"
    ]

    # 3. Make predictions
    predicted_labels, _ = model.predict(clothing_descriptions)

    # 4. Display results with explanations
    print("\nClothing Category Predictions:")
    print("=" * 50)
    for desc, label_num in zip(clothing_descriptions, predicted_labels):
        category = CLOTHING_CATEGORIES.get(label_num, "Unknown Category")
        print(f"Description: {desc}")
        print(f"Predicted Category: {category} (Class {label_num})")
        print("-" * 50)

except Exception as e:
    print(f"Error: {e}")

Model loaded successfully!


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Clothing Category Predictions:
Description: Lightweight cotton t-shirt with short sleeves perfect for hot weather
Predicted Category: Winter Wear (Class 1)
--------------------------------------------------
